In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/train_bogie_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195822 entries, 0 to 195821
Data columns (total 34 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   timestamp                        195822 non-null  object 
 1   train_id                         195822 non-null  int64  
 2   bogie_id                         195822 non-null  int64  
 3   track_gradient                   195822 non-null  float64
 4   speed_kmh                        195822 non-null  float64
 5   load_tons                        195822 non-null  float64
 6   external_temp_c                  195822 non-null  float64
 7   humidity_pct                     195822 non-null  float64
 8   days_since_inspection            195822 non-null  int64  
 9   vibration_x_rms                  195822 non-null  float64
 10  vibration_y_rms                  195822 non-null  float64
 11  vibration_z_rms                  195822 non-null  float64
 12  bo

In [3]:

# Asegurarnos de que timestamp es datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 1. Calcular cortes 70% y 85%
p70 = df['timestamp'].quantile(0.70)
p85 = df['timestamp'].quantile(0.85)

print("Corte 70%:", p70)
print("Corte 85%:", p85)

# 2. Crear particiones respetando el orden temporal
train = df[df['timestamp'] <= p70].reset_index(drop=True)
valid = df[(df['timestamp'] > p70) & (df['timestamp'] <= p85)].reset_index(drop=True)
test  = df[df['timestamp'] > p85].reset_index(drop=True)

print("Train shape:", train.shape)
print("Valid shape:", valid.shape)
print("Test shape :", test.shape)

print("Ratio fallo train :", train['target_fault'].mean())
print("Ratio fallo valid :", valid['target_fault'].mean())
print("Ratio fallo test  :", test['target_fault'].mean())


Corte 70%: 2024-04-07 05:26:42
Corte 85%: 2024-04-28 01:23:51
Train shape: (137075, 34)
Valid shape: (29373, 34)
Test shape : (29374, 34)
Ratio fallo train : 0.014984497537844246
Ratio fallo valid : 0.01518401252851258
Ratio fallo test  : 0.014774971062844693


In [4]:
train.to_csv("../data/bogie_train.csv", index=False)
valid.to_csv("../data/bogie_valid.csv", index=False)
test.to_csv("../data/bogie_test.csv", index=False)


## Train Undersampling

## Objetivo del train undersampling

Queremos corregir parcialmente el desbalance de clases en el conjunto de entrenamiento, donde solo ~1.5% de los registros son fallos (`target_fault = 1`). Si entrenáramos directamente sobre estos datos, muchos modelos tenderían a predecir casi siempre “no fallo”, porque los ejemplos de fallo son muy pocos en comparación.

---

## Qué vamos a hacer exactamente

### Separar por clases dentro de train

- **Conjunto `train_pos`**: todas las filas donde `target_fault = 1` (fallos reales).
- **Conjunto `train_neg`**: todas las filas donde `target_fault = 0` (no fallos).

---

### Aplicar undersampling solo a la clase mayoritaria (`train_neg`)

Tomaremos una muestra aleatoria de los no fallos, sin reemplazo, reduciendo su número para acercarnos a un ratio más manejable, por ejemplo:

- **Ratio 1:5** → por cada 1 fallo, mantener 5 no fallos.
- **Ratio 1:10** → por cada 1 fallo, mantener 10 no fallos.

Este ratio se elige como compromiso entre:

- No perder demasiada información de la clase 0.
- Dar suficiente “peso” a la clase 1 para que el modelo aprenda a distinguirla.

---

### Construir `train_under`

Pasos:

1. Unir `train_pos` (completo) + la muestra reducida de `train_neg`.
2. Mezclar las filas (*shuffle*) para que el modelo no vea primero todos los fallos juntos.

---

### Mantener `valid` y `test` sin tocar

No aplicamos undersampling en `valid` ni en `test`: esos conjuntos deben conservar el desbalance real para que las métricas de evaluación reflejen el comportamiento del modelo en producción.

---

## Ventajas y limitaciones

### Ventajas

- Aumenta la sensibilidad del modelo hacia la clase de fallo sin “inventar” datos (a diferencia del *oversampling* sintético).
- Reduce el tiempo de entrenamiento al trabajar con menos ejemplos de la clase 0.

### Limitaciones

- Se descarta información de la clase mayoritaria; por eso no conviene ir a un ratio 1:1 agresivo, sino a algo moderado (1:5, 1:10).
- Aun con undersampling, seguiremos necesitando pesos de clase y ajuste de umbral, porque el desbalance en `valid`/`test` sigue siendo fuerte.


In [5]:
# Contar ejemplos por clase en train
train_counts = train['target_fault'].value_counts()
n_neg = train_counts[0.0]
n_pos = train_counts[1.0]

print("No fallos (0):", n_neg)
print("Fallos (1):   ", n_pos)
print("Ratio original 0/1:", n_neg / n_pos)


No fallos (0): 135021
Fallos (1):    2054
Ratio original 0/1: 65.73563777994158


In [6]:
ratio = 5  # 1:5
n_neg_sample = int(ratio * n_pos)

print("Negativos que mantendremos:", n_neg_sample)


Negativos que mantendremos: 10270


In [7]:
# Separar por clase
train_pos = train[train['target_fault'] == 1.0]
train_neg = train[train['target_fault'] == 0.0]

# Muestreo aleatorio de la clase 0
train_neg_sample = train_neg.sample(
    n=n_neg_sample,
    random_state=42
)

# Unir y barajar
train_under = pd.concat([train_pos, train_neg_sample], axis=0)
train_under = train_under.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Comprobar nuevo balance
print("Shape train_under:", train_under.shape)
print(train_under['target_fault'].value_counts())
print("Nuevo ratio 0/1:", train_under['target_fault'].value_counts()[0.0] /
                         train_under['target_fault'].value_counts()[1.0])


Shape train_under: (12324, 34)
target_fault
0.0    10270
1.0     2054
Name: count, dtype: int64
Nuevo ratio 0/1: 5.0


In [8]:
train_under.to_csv("..//data/bogie_train_under_1a5.csv", index=False)


In [9]:
# 1. Definir columnas a excluir de las features
cols_to_exclude = [
    'timestamp',
    'train_id',
    'bogie_id',
    'target_fault',   # objetivo
    'alarm_level',    # pseudo-target
    'fault_type'      # pseudo-target / objetivo secundario
]

# 2. Construir lista de columnas de entrada (features)
all_cols = train_under.columns.tolist()
feature_cols = [c for c in all_cols if c not in cols_to_exclude]

print("Nº de features:", len(feature_cols))
print("Algunas features:", feature_cols[:10])

# 3. Crear X e y para cada conjunto
X_train_under = train_under[feature_cols].copy()
y_train_under = train_under['target_fault'].copy()

X_valid = valid[feature_cols].copy()
y_valid = valid['target_fault'].copy()

X_test = test[feature_cols].copy()
y_test = test['target_fault'].copy()

# 4. Comprobaciones rápidas
print("X_train_under:", X_train_under.shape, "y_train_under:", y_train_under.shape)
print("X_valid      :", X_valid.shape,       "y_valid      :", y_valid.shape)
print("X_test       :", X_test.shape,        "y_test       :", y_test.shape)

print("Ratio fallo train_under:", y_train_under.mean())
print("Ratio fallo valid      :", y_valid.mean())
print("Ratio fallo test       :", y_test.mean())


Nº de features: 28
Algunas features: ['track_gradient', 'speed_kmh', 'load_tons', 'external_temp_c', 'humidity_pct', 'days_since_inspection', 'vibration_x_rms', 'vibration_y_rms', 'vibration_z_rms', 'bogie_temp_c']
X_train_under: (12324, 28) y_train_under: (12324,)
X_valid      : (29373, 28) y_valid      : (29373,)
X_test       : (29374, 28) y_test       : (29374,)
Ratio fallo train_under: 0.16666666666666666
Ratio fallo valid      : 0.01518401252851258
Ratio fallo test       : 0.014774971062844693


## Renovación particiones con nuevas features (ventanas)

In [5]:
# 1) Cargar dataset con las nuevas features de ventana
df = pd.read_csv("../data/bogie_full_with_windows.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Asegurar orden temporal por tren y bogie
df = df.sort_values(["train_id", "bogie_id", "timestamp"]).reset_index(drop=True)

print("Shape df:", df.shape)
print("Ratio fallo global:", df["target_fault"].mean())

Shape df: (195822, 46)
Ratio fallo global: 0.014982994760547845


In [6]:
# 2) Calcular cortes temporales 70% y 85%
p70 = df["timestamp"].quantile(0.70)
p85 = df["timestamp"].quantile(0.85)

print("Corte 70%:", p70)
print("Corte 85%:", p85)

Corte 70%: 2024-04-07 05:26:42
Corte 85%: 2024-04-28 01:23:51


In [7]:
# 3) Crear nuevas particiones train / valid / test
train = df[df["timestamp"] <= p70].reset_index(drop=True)
valid = df[(df["timestamp"] > p70) & (df["timestamp"] <= p85)].reset_index(drop=True)
test  = df[df["timestamp"] > p85].reset_index(drop=True)

print("Train shape:", train.shape, "ratio fallo:", train["target_fault"].mean())
print("Valid shape:", valid.shape, "ratio fallo:", valid["target_fault"].mean())
print("Test  shape:", test.shape,  "ratio fallo:", test["target_fault"].mean())


Train shape: (137075, 46) ratio fallo: 0.014984497537844246
Valid shape: (29373, 46) ratio fallo: 0.01518401252851258
Test  shape: (29374, 46) ratio fallo: 0.014774971062844693


In [8]:
# 4) Crear train_under con ratio 1:5 (mantener todos los 1, muestrear 0)
train_counts = train["target_fault"].value_counts()
n_neg = train_counts[0.0]
n_pos = train_counts[1.0]

ratio = 5  # 1:5
n_neg_sample = int(ratio * n_pos)

print("No fallos (0):", n_neg)
print("Fallos (1):   ", n_pos)
print("Negativos que se mantendrán:", n_neg_sample)

train_pos = train[train["target_fault"] == 1.0]
train_neg = train[train["target_fault"] == 0.0]

train_neg_sample = train_neg.sample(n=n_neg_sample, random_state=42)

train_under = pd.concat([train_pos, train_neg_sample], axis=0)
train_under = train_under.sample(frac=1.0, random_state=42).reset_index(drop=True)

print("Shape train_under:", train_under.shape)
print(train_under["target_fault"].value_counts())
print("Nuevo ratio 0/1:", train_under["target_fault"].value_counts()[0.0] /
                         train_under["target_fault"].value_counts()[1.0])


No fallos (0): 135021
Fallos (1):    2054
Negativos que se mantendrán: 10270
Shape train_under: (12324, 46)
target_fault
0.0    10270
1.0     2054
Name: count, dtype: int64
Nuevo ratio 0/1: 5.0


In [10]:
# 5) Guardar las nuevas particiones con sufijo _windows
train.to_csv("../data/bogie_train_windows.csv", index=False)
valid.to_csv("../data/bogie_valid_windows.csv", index=False)
test.to_csv("../data/bogie_test_windows.csv", index=False)
train_under.to_csv("../data/bogie_train_under_1a5_windows.csv", index=False)
